In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Se levanta el split hecho en el NoteBook anterior

In [2]:
entrenarX_train = pd.read_csv("X_train")
X_train.head()

,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,anio_publ
0,0.0,3.0,2.0,3.0,196.0,160.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2014
1,20.0,4.0,2.0,2.0,245.0,196.0,104734.0,NaN,NaN,0.0,0.0,0.0,1.0,0.0,2012
2,5.0,3.0,1.0,2.0,108.0,120.0,58952.0,NaN,NaN,0.0,0.0,0.0,1.0,1.0,2015
3,10.0,3.0,1.0,3.0,200.0,147.0,49066.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,2016
4,NaN,NaN,0.0,NaN,NaN,250.0,24892.0,19.23339,-99.166991,0.0,0.0,0.0,0.0,0.0,2016


In [3]:
X_test = pd.read_csv("X_test")

In [4]:
y_train = pd.read_csv("y_train")

In [5]:
y_test = pd.read_csv("y_test")

# Se levantan todos los XGBoost guardados

In [6]:
xgb_created = []

## Atencion! n tiene que coincidir con la cantidad guardada!

In [7]:
n = 200

In [8]:
for i in range(n):
    current_xgb = xgb.XGBRegressor()
    current_xgb.load_model("xgb_{}".format(i))
    xgb_created.append(current_xgb)

# Se arma el DF de precios para entrenar la NN

In [9]:
NN_train = pd.DataFrame(index = X_train.index.copy())

In [10]:
for i in range(n):
    current_price_pred = xgb_created[i].predict(X_train)
    NN_train['pred_{}'.format(i)] = current_price_pred
    print("xgb_{} finished pred".format(i))
NN_train.head()

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,...,pred_190,pred_191,pred_192,pred_193,pred_194,pred_195,pred_196,pred_197,pred_198,pred_199
0,1675568.750,2279996.000,2.278069e+06,1962261.875,1908726.000,1975850.125,2.041407e+06,2135030.50,1780616.500,2251092.750,...,2.332686e+06,2058865.25,1.892811e+06,2.447693e+06,1982648.375,2.459340e+06,2373884.500,1711514.375,2.224779e+06,2.174826e+06
1,1133347.250,1877194.375,1.398618e+06,2622835.500,1574958.500,2191358.000,1.781564e+06,1873575.75,1501947.625,2166757.250,...,1.423210e+06,1886828.50,1.360701e+06,1.722803e+06,1879809.250,1.749367e+06,1678725.625,1802272.750,1.506787e+06,1.625895e+06
2,1297917.750,872463.250,9.312116e+05,935558.625,877573.375,1290895.750,9.639478e+05,1374816.50,1059143.125,1064682.625,...,8.692382e+05,891783.00,1.043634e+06,9.136286e+05,810946.500,8.736514e+05,1120368.125,901425.625,1.118626e+06,9.779751e+05
3,1428815.125,343955.125,1.164508e+06,1830824.250,1702865.375,1664202.375,1.508122e+06,1421637.00,1970687.750,1090036.625,...,1.471208e+06,1881730.50,1.671257e+06,1.818161e+06,2337558.500,2.057019e+06,1866981.375,2152244.750,1.691376e+06,1.937423e+06
4,2332554.250,3657671.500,1.526109e+06,1624921.625,2327190.750,1472552.000,2.774342e+06,2749102.00,3642267.000,4184341.250,...,2.270237e+06,2076781.00,1.458910e+06,1.402020e+06,3152021.250,3.898565e+05,2001380.750,1866278.500,-1.618149e+05,2.222274e+06


In [11]:
len(NN_train)

168000

In [12]:
len(NN_train)

168000

# Se Escala el NN_train

In [2]:
scaler = StandardScaler()

In [3]:
scaler.fit(NN_train)

NameError: name 'NN_train' is not defined

In [ ]:
NN_train = scaler.transform(NN_train)

## Se exporta el NN_train para poder buscar hiperparametros

In [13]:
NN_train.to_csv("NN_train", index = False)

# Se define el modelo 

sdg o adam como solver (lbfg queda descartado porque el train es muy grande).
Supuestamente sdg es mejor si el modelo esta bien tuneado.

In [ ]:
MLP_reg = MLPRegressor(activation = 'relu', solver = 'sdg', , verbose = 1, alpha = 0.001, random_state = 0)

# Se entrena el modelo